In [1]:
### imports
import sys
sys.path.append('..')

import joblib
import pandas as pd
from src.dashboardScraper import DashboardScraper
from src.arbitrageOptimizer import ArbitrageOptimizer

In [2]:
### load sample data
optimizer = ArbitrageOptimizer()
games = joblib.load('../data/sample.dat')

In [19]:
### show table of avaiable games
from bokeh.io import show, output_file
from bokeh.layouts import widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, Panel, TableColumn, Tabs

In [36]:
### create table of the current opportunities
def get_data_table_from_games(games):
    optimizer = ArbitrageOptimizer()
    data_match = []
    data_league = []
    data_result_to_bet = []
    data_date = []
    data_time_to_match = []
    data_best_bookie = []
    data_best_odds = []
    data_mean_median = []
    data_ratio = []

    for game in games:
        data_match.append(game['game']['match_title'])
        data_league.append(game['game']['league'])
        data_result_to_bet.append(game['game']['result_to_bet'])
        data_date.append(game['game']['date'])
        data_time_to_match.append(game['game']['time_to_match'])
        data_best_bookie.append(game['game']['best_bookie'])
        data_best_odds.append(game['game']['best_odds'])
        data_mean_median.append(str(game['game']['mean']) + "/" + str(game['game']['mean']))
        optimizer.load_game(game)
        data_ratio.append(round(100*optimizer.get_optimal_ratio(),1))
    
    data = dict(
        match = data_match, 
        league = data_league, 
        result_to_bet = data_result_to_bet,
        date = data_date,
        time_to_match = data_time_to_match,
        best_bookie = data_best_bookie,
        best_odds = data_best_odds,
        mean_median = data_mean_median,
        ratio = data_ratio,
    )
    
    source = ColumnDataSource(data)
    
    columns = [
        TableColumn(field="match", title="Mitch Title"),
        TableColumn(field="league", title="League"),
        TableColumn(field="result_to_bet", title="Result to Bet"),
        TableColumn(field="date", title="Date"),
        TableColumn(field="time_to_match", title="Time to Match"),
        TableColumn(field="best_bookie", title="Best Bookie"),
        TableColumn(field="best_odds", title="Best Odds"),
        TableColumn(field="mean_median", title="Mean/Median"),
        TableColumn(field="ratio", title="Ratio to Bet (%)"),
    ]
    
    data_table = DataTable(source=source, columns=columns, width=1400, height=1000, sizing_mode='stretch_both')
    return data_table

data_table = get_data_table_from_games(games)
show(widgetbox(data_table))

In [7]:
### create DataTable for arbitrage opportunity
def get_arbitrage_table(game):
    optimizer = ArbitrageOptimizer()
    optimizer.load_game(game)
    arbitrage_opportunity = optimizer.get_arbitrage_opportunity()
    
    if arbitrage_opportunity['arbitrage'] != 'yes':
        return None
    else:
        column_1 = [
            arbitrage_opportunity['bet']['1']['bookie'],
            arbitrage_opportunity['bet']['1']['odds'],
            str(round(100*arbitrage_opportunity['bet']['1']['bet_size'],1)),
        ]
        column_X = [
            arbitrage_opportunity['bet']['X']['bookie'],
            arbitrage_opportunity['bet']['X']['odds'],
            str(round(100*arbitrage_opportunity['bet']['X']['bet_size'],1)),
        ]
        column_2 = [
            arbitrage_opportunity['bet']['2']['bookie'],
            arbitrage_opportunity['bet']['2']['odds'],
            str(round(100*arbitrage_opportunity['bet']['2']['bet_size'],1)),
        ]

        data = dict(column_1 = column_1, column_X = column_X, column_2 = column_2,)
        source = ColumnDataSource(data)

        columns = [
            TableColumn(field="column_1", title="1"),
            TableColumn(field="column_X", title="X"),
            TableColumn(field="column_2", title="2"),
        ]

        data_table = DataTable(source=source, columns=columns, width=500, height=200)
        return data_table, arbitrage_opportunity['profit']

In [31]:
### create separate tabs for the arbitrage opportunities
arbitrage_tables = []
for game in games:
    output = get_arbitrage_table(game)
    if output is not None:
        arbitrage_tables.append(output)
# sort tables in descending order of the profit
arbitrage_tables.sort(key=lambda x : x[1], reverse=True)

# create a separate tab for each arbitrage table
tabs = []
for arbitrage_table in arbitrage_tables:
    profit = str(round(100*arbitrage_table[1],1))
    tabs.append(Panel(child=arbitrage_table[0], title="Profit : {}".format(profit)))

show(Tabs(tabs=tabs))